In [1]:
import pandas as pd
from datetime import date, timedelta
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [2]:
day = date(2024, 3, 1)
day_dateoffset = day - pd.DateOffset(months=1)
print(day_dateoffset)

2024-02-01 00:00:00


In [3]:
day = date(2024, 3, 1)
day_timedelta = day - timedelta(days=31)
print(day_timedelta)

2024-01-30


In [4]:
f = open('call_rate.csv', 'r', encoding='utf-8')

# 데이터를 완전히 동일하게 가져올 수 있는가
headers = f.readline().rstrip() # 헤더 읽기, 맨 끝 줄바꿈기호 삭제
print(headers.split('\t'))
headers = headers.split('\t') # 같은 이름의 변수에 리스트로 바꾼 것 넣어주기

# 데이터 읽기
lines = f.readlines()
print(lines[:5])

new_datas = []
for line in lines: # 한 줄씩 
    data = line.rstrip().split('\t') # 줄바꿈 기호 지우고, ,로 잘라준다.
    dict_data = { header: data[i] for i, header in enumerate(headers) }
    new_datas.append(dict_data) # 데이터를 딕셔너리로 만들어서 넣어야된다.

f.close()

['날짜', '종가', '등락률']
['2024-03-21\t3.5290\t-0.11\n', '2024-03-20\t3.5330\t0.56\n', '2024-03-19\t3.5130\t2.09\n', '2024-03-18\t3.4410\t-0.66\n', '2024-03-15\t3.4640\t-0.63\n']


In [5]:
def find_nearest_date(df, target_date):
    delta = pd.Timedelta(days=1)
    nearest_date = None,
    min_difference = pd.Timedelta.max

    for date in df['date']:
        difference = abs(target_date - date)
        if difference < min_difference:
            min_difference = difference
            nearest_date = date

    return nearest_date

def label_call(df):
    df_ud = pd.DataFrame(index=df.index, columns=['date', 'price', 'label'])

    # 'date' 열을 datetime 형식으로 변환
    df['date'] = pd.to_datetime(df['date'])

    for i, row in df.iterrows():
        time = row['date']
        time2 = time - pd.DateOffset(months=1)

        # 이전 달의 날짜가 인덱스에 있는지 확인
        if time2 in df['date'].values:
            prev_row = df[df['date'] == time2].iloc[0]
        else:
            # 이전 날짜가 없는 경우 가장 가까운 날짜로 설정
            nearest_date = find_nearest_date(df, time2)
            prev_row = df[df['date'] == nearest_date].iloc[0]

        if row['price'] > prev_row['price']:
            label = '1'
        elif row['price'] == prev_row['price']:
            label = 0
        else:
            label = '-1'

        df_ud.loc[i] = [time.date(), row['price'], label]

    return df_ud 

In [6]:
df = pd.DataFrame(new_datas)

call = label_call(df)
call

KeyError: 'date'

In [ ]:
pip install ekonlpy

In [45]:
import konlpy as kn
import MeCab as mc
import ekonlpy as ek
import pandas as pd
import numpy as np
import re
from pandas import Series, DataFrame 

In [46]:
# 크롤링 결과물 불러오기 - 이데일리 뉴스 (4분 소요)
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK
mpck = MPCK()

# 패턴 정의
news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/edaily_news.json'
df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False) # 날짜별 내림차순

# 특정패턴 제거
for pattern in news_pattern:
    df['content'] = df['content'].apply(lambda x: re.sub(pattern, '', x))
news1_df = df

news1_df['words']=" "  # 빈 값을 가진 words column 추가
for i, row in df.iterrows() :
    try :
        tmp_df = news1_df['content']
        #임시 데이터프레임에 불러온 기사의 본문 내용 리스트로 저장
        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
    
        df.loc[i, 'words'] = row['words']
        #eKoNLPy로 tokenizing 한 것을 데이터프레임에 column 추가

    except Exception as ex :
        print("fail")
        
news1_df = news1_df.drop(['content', 'title'], axis=1)  # axis=1은 열을 의미함
news1_df.columns

Index(['date', 'words'], dtype='object')

In [47]:
# 크롤링 결과물 불러오기 - 중앙일보 뉴스 (8분 소요)
mpck = MPCK()

news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/joongamg_news.json'
df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False)

for pattern in news_pattern:
    df['content'] = df['content'].apply(lambda x: re.sub(pattern, '', x))
news2_df = df

news2_df['words']=" "
for i, row in df.iterrows() :
    try :
        tmp_df = news2_df['content']

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']

    except Exception as ex :
        print("fail")

news2_df = news2_df.drop(['content', 'title'], axis=1)

In [11]:
a = '안녕하세요'

a.split('하')

print(a)

안녕하세요


In [48]:
# 크롤링 결과물 불러오기 - 머니투데이 뉴스 (3분 30초)
mpck = MPCK()

news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/moneytoday_news.json'
df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False) 

for pattern in news_pattern:
    df['content'] = df['content'].apply(lambda x: re.sub(pattern, '', x))
news3_df = df

news3_df['words']=" " 
for i, row in df.iterrows() :

    try :
        tmp_df = news3_df['content']

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']

    except Exception as ex :
        print("fail")

news3_df = news3_df.drop(['content', 'title'], axis=1)

In [49]:
# 크롤링 결과물 불러오기 - 의사록 
mpck = MPCK()

news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/minutes_re.tsv'
df = pd.read_csv(result_path, sep='\t')

for pattern in news_pattern:
    df['내용'] = df['내용'].apply(lambda x: re.sub(pattern, '', x))
min_df = df

min_df['words']=" "
for i, row in df.iterrows() :
    try :
        tmp_df = min_df['내용']

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']

    except Exception as ex :
        print("fail")

min_df = min_df.drop('내용', axis=1) 
min_df = min_df.rename(columns={'날짜': 'date'})

In [50]:
# json 파일 구조가 달라져서 수정하고 진행
# import json

# # JSON 파일 읽기
# with open('./crawl_result/bone_report1.json', 'r') as file:
#     data = json.load(file)

# # 새로운 키와 값을 포함하는 리스트 생성
# new_data_list = []
# for key, value in data.items():
#     new_data = {"date": key, "text": value}
#     new_data_list.append(new_data)

# with open('new_data2.json', 'w', encoding='utf-8') as file:
#     json.dump(new_data_list, file, indent=4, ensure_ascii=False)

In [51]:
# 크롤링 결과물 불러오기 - 채권 보고서 
mpck = MPCK()

news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자', '전일', '채권시장']
bone_pattern = []

result_path = './crawl_result/bone_report.json'
df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False)

for pattern in news_pattern:
    df['text'] = df['text'].apply(lambda x: re.sub(pattern, '', x))
bond_df = df


bond_df['words']=" "
for i, row in df.iterrows() :
    try :
        tmp_df = bond_df['text']

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']

    except Exception as ex :
        print("fail")

bond_df = bond_df.drop('text', axis=1)

index_to_drop = 1179 # date가 없어서 오류나는 인덱스 삭제
bond_df.drop(index_to_drop, inplace=True)

In [53]:
# 전체 크롤링 데이터들 -> 데이터프레임으로 합치기
crawl = pd.concat([news1_df, news2_df, news3_df, min_df, bond_df])
crawl['date'] = pd.to_datetime(crawl['date']).dt.date
crawl

,date,words
7952,2024-03-21,마이너스금리/NNG.해제/NNG.엔/NNG.통화/NNG.대비/NNG.약세/NNG.보...
7947,2024-03-21,대해/VV.전기차/NNG.적용/NNG.하/VV.있/VV.고압/NNG.적층세라믹콘덴서...
7956,2024-03-21,국고채/NNG.시장/NNG.간밤/NNG.흐름/NNG.반영/NNG.하/XSV.강세/N...
7953,2024-03-21,달러/NNG.원대/NNG.하락/NNG.출발/NNG.하/VV.fed/NNG.fed/N...
7943,2024-03-21,지수/NNG.현지/NNG.일제히/MAG.상승/NNG.하/VV.다우존스산업평균지수/N...
...,...,...
2,2008-05-04,금리/NNG.인하/NNG.요구/NNG.부응/NNG.스쳐/VV.나타나/VV.금리/NN...
1,2008-04-28,대내외/NNG.요인/NNG.금리/NNG.상반/NNG.되/VV.영향/NNG.미치/VV...
20,2008-04-07,하단/NNG.상단/NNG.테스트/NNG.가능성/NNG.무게/NNG.글로벌/NNG.신...
0,2008-04-01,동향/NNG.강세/NNG.흐름/NNG.유지/NNG.되/XSV.환율/NNG.정책/NN...


In [54]:
# # korean stop words 불러오기
# stopwords_path = 'korean_sotp_words.txt'
# stopwords = []
# with open(stopwords_path, 'r', encoding='utf-8') as file:
#     for line in file:
#         stopwords.append(line.strip())
# print(stopwords)  # 불용어 모음

In [55]:
# word = []
# for tag in text: #토큰화된 text
#     if tag[1] not in stopwords:
#         if tag[0] not in stopwords:
#             word.append(tag[0])

In [56]:
# 품사 태깅 제거
import re

def extract_hangul_words(text):
    words = ' '.join(re.findall('[가-힣]+', text))
    return words

crawl['words'] = crawl['words'].apply(extract_hangul_words)

In [57]:
# 콜금리에 date기준으로 크롤링 칼럼 추가하기
import pandas as pd

# Date 열을 datetime으로 변환하고 인덱스로 설정
call['date'] = pd.to_datetime(call['date'])
call.set_index('date', inplace=True)

crawl['date'] = pd.to_datetime(crawl['date'])
crawl.set_index('date', inplace=True)

# 중복된 인덱스 제거
call = call[~call.index.duplicated()]
crawl = crawl[~crawl.index.duplicated()]

# 동일한 날짜에 대해 열을 합치기
merged_df = pd.concat([call, crawl], axis=1)

# 'date' 열을 다시 열로 변환
merged_df.reset_index(inplace=True)

# 결과 출력
print(merged_df.columns)
print(merged_df)

Index(['date', 'price', 'label', 'words'], dtype='object')
           date price label                                              words
0    2006-05-11   NaN   NaN  조 의거 하 의안 통화정책 방향 상정 하 토의 내용 국내외 경제동향 하 국제 유가 ...
1    2006-05-25   NaN   NaN  담당 보가 지급 결제 원활 위하 금융기관 일중 당좌대출 한도규제 폐지 하 일중 당좌...
2    2006-06-08   NaN   NaN  조 의거 하 의안 통화정책 방향 상정 하 토의 내용 국내외 경제동향 하 경기 선행 ...
3    2006-06-22   NaN   NaN  조건 구비 하 사항 명시 하 차액 결제 참가 기관 자금 조정 적수 보전 이자 정산 ...
4    2006-07-07   NaN   NaN  보 금융시장 동향 대하 보고 내용 별첨 조 의거 하 의안 통화정책 방향 상정 하 토...
...         ...   ...   ...                                                ...
5255 2024-03-22   NaN   NaN  요즘 상당 요금 고지서 한숨 나오 넘 마땅 일자리 구하 못하 건설현장 일용직 하 부...
5256 2024-03-24   NaN   NaN  있 건물 외경 고금리 작별 다가섰 전세계 금리 결정 금리 위크 요약 결과 다 고물가...
5257 2024-03-25   NaN   NaN  청계 견본주택 시민 단지 모형 살펴보 뉴스 청약 홈 개편 달 문 닫 분양시장 재개 ...
5258 2024-03-26   NaN   NaN  동향 약보합 마감 대외 요인 연동 되 흐름 지속 특별 이벤트 부재 속 매수 심리 이...
5259        NaT   NaN   NaN                                             

In [58]:
# 'label' 칼럼의 값이 '1(매파)'인 행과 '-1(비둘기파)'인 행으로 데이터프레임 분리
df_P = merged_df[merged_df['label'] == '1']
df_N = merged_df[merged_df['label'] == '-1']

# Nan인 행 제거
df_P = df_P.dropna()
df_N = df_N.dropna()

# 필요없는 열 제거 
df_P = df_P.drop(['date', 'price'], axis=1)
df_N = df_N.drop(['date', 'price'], axis=1)

word_df = pd.concat([df_P,df_N],axis=0)
x_tain = word_df["words"]
y_tain = word_df["label"]

In [60]:
# # 매파 words 빈도수 체크 
# import pandas as pd
# from nltk.tokenize import word_tokenize
# from collections import Counter

# # 텍스트 칼럼에서 단어 횟수 세기
# text = ' '.join(df_P['words'])  # 텍스트 칼럼의 모든 텍스트를 하나의 문자열로 결합
# tokens = word_tokenize(text)  # 텍스트를 단어로 토큰화
# word_counts = Counter(tokens)  # 단어 등장 횟수 카운트

# # 빈도수가 15 이상인 단어 필터링
# filtered_word_counts = {word: count for word, count in word_counts.items() if count >= 15}

# df_P_counts = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
# df_P_counts['P'] = 1 # 라벨링 다시 추가

In [61]:
# # 비둘기파 words 빈도수 체크
# text = ' '.join(df_N['words'])  # 텍스트 칼럼의 모든 텍스트를 하나의 문자열로 결합
# tokens = word_tokenize(text)  # 텍스트를 단어로 토큰화
# word_counts = Counter(tokens)  # 단어 등장 횟수 카운트

# # 빈도수가 15 이상인 단어 필터링
# filtered_word_counts = {word: count for word, count in word_counts.items() if count >= 15}

# # 데이터프레임 생성
# df_N_counts = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
# df_N_counts['N'] = 1

In [62]:
# df_B_counts.to_csv("dovish_count_1.csv", mode='w')

In [91]:
# P 1-gram
from collections import defaultdict
import pandas as pd

def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        words = row.split()
        ngrams.extend([' '.join(words[i:i+n]) for i in range(len(words)-n+1)])
    return ngrams

def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

n = 1
ngrams = generate_ngrams(df_P['words'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 15 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 30}

# 데이터프레임으로 변환
P_1gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
P_1gram_df['P'] = 1
P_1gram_df 

,ngram,frequency,P
0,약세,506,1
1,영향,884,1
2,인하,1742,1
3,진행,412,1
4,되,8059,1
...,...,...,...
1274,대환,32,1
1275,코로나,333,1
1276,지사,31,1
1277,팬데믹,50,1


In [88]:
# N 1-gram
from collections import defaultdict
import pandas as pd

def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        words = row.split()
        ngrams.extend([' '.join(words[i:i+n]) for i in range(len(words)-n+1)])
    return ngrams

def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

n = 1
ngrams = generate_ngrams(df_N['words'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 15 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 5}

# 데이터프레임으로 변환
N_1gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
N_1gram_df['N'] = -1
N_1gram_df 

,ngram,frequency,N
0,강세,355,-1
1,우호적,50,-1
2,대내외,50,-1
3,여건,193,-1
4,인하,1777,-1
...,...,...,...
4065,건전재정,5,-1
4066,워크아웃,7,-1
4067,장마감후,6,-1
4068,부서장,6,-1


In [93]:
merged_df = pd.merge(P_1gram_df , N_1gram_df , on='ngram', how='inner')
merged_df = merged_df.drop(['P', 'N'], axis=1) 
merged_df = merged_df.rename(columns={'frequency_x': 'P_count'})
merged_df = merged_df.rename(columns={'frequency_y': 'N_count'})
merged_df.set_index('ngram', inplace=True)
merged_df

,P_count,N_count
ngram,,
약세,506,307
영향,884,667
인하,1742,1777
진행,412,229
되,8059,6489
...,...,...
대환,32,14
코로나,333,407
지사,31,7


In [95]:
# merged_df.to_csv("final_count_1.csv", mode='w')

#### ----------------------------------------------------------- 2_gram ------------------------------------------------------------------

In [24]:
# 2-gram
from collections import defaultdict
import pandas as pd

def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        words = row.split()
        ngrams.extend([' '.join(words[i:i+n]) for i in range(len(words)-n+1)])
    return ngrams

def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

n = 2
ngrams = generate_ngrams(df_P['words'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 15 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 15}

# 데이터프레임으로 변환
selected_ngrams_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
selected_ngrams_df['N'] = 1
selected_ngrams_df 

,ngram,frequency,N
0,순매수 상위,44,1
1,순매도 상위,44,1
2,상위 보험기금,22,1
3,상위 투신,22,1
4,상위 외국인,22,1
...,...,...,...
3083,물가 둔화,17,1
3084,목 목,18,1
3085,리 오프닝,33,1
3086,인플레이션 둔화,31,1
